In [ ]:
import pandas as pd
df=pd.read_csv('Processed_data.csv')
city=df['Worksite'].apply(lambda x: x.split(', ')[0])
df['city']=city
latlon=pd.read_csv('place_data.csv')
latlon['city']=latlon['place'].apply(lambda x: x.split(', ')[0])
# Merging the dataframes
merged_df = pd.merge(df, latlon, left_on='city', right_on='city', how='inner')

final_df=merged_df.drop(['Unnamed: 0', 'place'], axis=1)

In [ ]:
final_df=final_df.dropna()



In [ ]:
def plot_city_density(df, title):
    from mpl_toolkits.basemap import Basemap
    from scipy.stats import gaussian_kde
    import numpy as np
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(figsize=(12, 6))

    map = Basemap(llcrnrlon=-119, llcrnrlat=22, urcrnrlon=-64, urcrnrlat=49,
                  projection='lcc', lat_1=32, lat_2=45, lon_0=-95)
    map.readshapefile('./st99_d00', name='states', drawbounds=True)

    city_coords = df.groupby('city')[['latitude', 'longitude']].mean()

    x = np.repeat(city_coords['longitude'].values, df['city'].value_counts().values)
    y = np.repeat(city_coords['latitude'].values, df['city'].value_counts().values)


    xy = np.vstack([x, y])
    z = gaussian_kde(xy)(xy)

    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]

    xmap, ymap = map(x, y)
    heatmap = ax.scatter(xmap, ymap, c=z, s=20, cmap='viridis')
    cb = plt.colorbar(heatmap)
    cb.set_label('Density')

    plt.title(title)
    plt.show()




In [ ]:
plot_city_density(final_df, 'H1B Petition Heatmap')